Se carga y selecciona datos de dos fuentes diferentes (una con los datos de las IMUs y otra con datos de la evaluación de los pacientes MDS UPDRS), se selecciona columnas específicas de los datos y se asigna los subconjuntos de datos resultantes a las variables data y patientsMds, respectivamente.

In [44]:
import numpy as np
import pandas as pd
from src.settings  import ROOT_DIR

data: pd.DataFrame = pd.read_pickle(ROOT_DIR / 'data' / 'processed' / 'anon_imu_data_time_series_cleaned.pkl')
used_columns = ['anon_id', 'date_measure', 'time_stamp', 'imu_accX_right', 'imu_accX_left','PD']
data = data[used_columns]
print(data.columns)
patientsMds:pd.DataFrame = pd.read_excel(ROOT_DIR / 'data' / 'raw' / 'patients_mdsupdrs_results.xlsx')
patientsMds_used_columns = ['Cédula','Temblor postural – Mano derecha','Temblor postural – Mano izquierda','Temblor de acción – Mano derecha','Temblor de acción – Mano izquierda']

Index(['anon_id', 'date_measure', 'time_stamp', 'imu_accX_right',
       'imu_accX_left', 'PD'],
      dtype='object')


Se crea un nuevo campo para no depender de la fecha para diferenciar cada muestra.

In [45]:
data['combined_id_date'] = data['anon_id'].astype(str) + '_' + data['date_measure'].astype(str)
print(data.tail())

       anon_id        date_measure  time_stamp  imu_accX_right  imu_accX_left  \
494723     179  2023-6-17-10-39-17     44757.0        8.601266       8.963518   
494724     179  2023-6-17-10-39-17     44798.0        7.113219       9.182925   
494725     179  2023-6-17-10-39-17     44839.0        5.800859       9.556166   
494726     179  2023-6-17-10-39-17     44880.0        5.117749       9.967539   
494727     179  2023-6-17-10-39-17     44900.0        5.303877      10.260220   

        PD        combined_id_date  
494723   0  179_2023-6-17-10-39-17  
494724   0  179_2023-6-17-10-39-17  
494725   0  179_2023-6-17-10-39-17  
494726   0  179_2023-6-17-10-39-17  
494727   0  179_2023-6-17-10-39-17  


Se realiza el cálculo de energía total por brazo.

In [47]:
df=data.copy()
# Agrupa los datos por 'date_measure'
grouped_data = df.groupby('combined_id_date')

# Inicializa listas para almacenar los resultados
energy_data = []

# Itera sobre cada grupo
for combined_id_date, group in grouped_data:
    # Obtén las señales de aceleración de ambos lados
    accX_right = group['imu_accX_right'].values
    accX_left = group['imu_accX_left'].values

    # Calcula la FFT de cada señal de aceleración
    fft_result_right = np.fft.fft(accX_right)
    fft_result_left = np.fft.fft(accX_left)

    # Calcula la magnitud de la FFT (utiliza np.abs para obtener el valor absoluto)
    fft_magnitude_right = np.abs(fft_result_right)
    fft_magnitude_left = np.abs(fft_result_left)

    # Calcula la energía total para cada acelerómetro
    total_energy_right = np.sum(fft_magnitude_right)
    total_energy_left = np.sum(fft_magnitude_left)

    # Almacena la energía total en la lista
    energy_data.append({'combined_id_date': combined_id_date, 'total_energy_right': total_energy_right, 'total_energy_left': total_energy_left})

# Convierte la lista de resultados en un DataFrame
energy_df = pd.DataFrame(energy_data)

# Imprime el DataFrame con la energía total por medida
print(energy_df.columns)

Index(['combined_id_date', 'total_energy_right', 'total_energy_left'], dtype='object')


Se imprime una muestra específica para observar los resultados

In [48]:
# Suponiendo que 'energy_df' es tu DataFrame con los resultados de energía
specific_combined_id_date = '10_2023-6-22-10-28-8'

# Filtra el DataFrame para obtener solo las filas correspondientes a la fecha específica
specific_specific_combined_id_date_data = energy_df.loc[energy_df['combined_id_date'] == specific_combined_id_date]

# Muestra los resultados
print(specific_specific_combined_id_date_data)


        combined_id_date  total_energy_right  total_energy_left
12  10_2023-6-22-10-28-8        85653.639433      115495.635742


Se cruzan las cédulas con las que ya se habían obtenido de anonimización para unir los dataframes.

In [50]:
from src.data.eda_helper import load_id_map
def get_anon_id(ced: str, id_map: pd.DataFrame = None) -> str:

    if ced is None or ced == "":
        return ""

    if id_map is None:
        id_map = load_id_map()
    real_id_list = id_map.loc[id_map['patient_id'] == ced, 'anon_id'].tolist()
    return real_id_list[0] if len(real_id_list) > 0 else ""

patientsMds = patientsMds[patientsMds_used_columns]
patientsMds['anon_id'] = patientsMds['Cédula'].astype(str).apply(get_anon_id)

patientsMds.head()



,Cédula,Temblor postural – Mano derecha,Temblor postural – Mano izquierda,Temblor de acción – Mano derecha,Temblor de acción – Mano izquierda,anon_id
0,4663131,0,1,0,0,11
1,25714829,1,1,0,0,87
2,10542415,1,0,0,0,32
3,10526776,0,0,0,0,27
4,25268997,1,1,0,0,78


Se borra la cédula para respetar privacidad de pacientes y re realiza un split para obtener las cédulas de los pacientes.

In [51]:
patientsMds.drop('Cédula', axis=1, inplace=True)

#Re-split para realizar el cruce por id
energy_df['id']=energy_df['combined_id_date'].str.split('_').str[0]


Unimos los dataframes por coincidencia en cédulas.

In [53]:
# Realizar la unión por las columnas 'id' y 'anon_id'
merged_df = pd.merge(energy_df, patientsMds, left_on='id', right_on='anon_id', how='inner')

# Eliminar la columna 'anon_id' si no es necesaria en el DataFrame final
merged_df.drop('anon_id', axis=1, inplace=True)
id_column = merged_df.pop('id')
merged_df.insert(1, 'id', id_column)

# Imprimir el DataFrame resultante para verificar
merged_df.head()

,combined_id_date,id,total_energy_right,total_energy_left,Temblor postural – Mano derecha,Temblor postural – Mano izquierda,Temblor de acción – Mano derecha,Temblor de acción – Mano izquierda
0,102_2023-8-31-15-3-10,102,75142.285996,81240.302861,3,1,0,0
1,102_2023-8-31-15-5-5,102,65801.483835,103992.161572,3,1,0,0
2,103_2023-7-11-10-15-46,103,13.412852,24.038894,0,0,0,0
3,104_2023-9-12-11-8-0,104,240563.682982,222068.000913,0,0,0,0
4,106_2023-2-23-15-52-11,106,103817.726651,101727.156213,0,0,0,0
